# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [19]:
pip install ipywidgets


Note: you may need to restart the kernel to use updated packages.


In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
import ipywidgets as widgets

# Import API key
from api_keys import g_key

In [2]:
### Store Part I results into DataFrame
#Load the csv exported in Part I to a DataFrame
weatherfile="output_data/cities.csv"
weatherfile_df= pd.read_csv(weatherfile, encoding="ISO-8859-1")
weatherfile_df.head(10)

,city_id,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,gorontalo,0.54,123.06,75.36,89,69,2.80,ID,1595256634
1,1,urumqi,43.80,87.60,82.40,34,100,4.47,CN,1595256634
2,2,saint george,37.10,-113.58,84.99,20,1,5.82,US,1595256634
3,3,bethel,41.37,-73.41,84.00,69,1,8.05,US,1595256634
4,4,east london,-33.02,27.91,73.40,25,0,5.82,ZA,1595256635
5,5,busselton,-33.65,115.33,50.00,60,100,10.18,AU,1595256635
6,6,ust-maya,60.42,134.53,68.68,61,100,14.38,RU,1595256635
7,7,henties bay,-22.12,14.28,77.50,23,0,9.24,NaN,1595256635
8,8,rikitea,-23.12,-134.97,68.47,57,85,13.09,PF,1595256635
9,9,codrington,-38.27,141.97,50.14,77,70,6.91,AU,1595256636


In [3]:
### Humidity Heatmap
gmaps.configure(api_key=g_key)
locations = weatherfile_df[["Lat", "Lng"]]
humidity = weatherfile_df["Humidity"]

* Use the Lat and Lng as locations and Humidity as the weight.

* Add Heatmap layer to map.


In [4]:
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

In [5]:
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, 
                                 max_intensity= 100, point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
narrowed_city_df = weatherfile_df.loc[(weatherfile_df["Wind Speed"] <= 10) & (weatherfile_df["Cloudiness"] == 0) & \
                                   (weatherfile_df["Max Temp"] >= 70) & (weatherfile_df["Max Temp"] <= 80)].dropna()

narrowed_city_df

,city_id,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,east london,-33.02,27.91,73.40,25,0,5.82,ZA,1595256635
52,52,guerrero negro,27.98,-114.06,72.84,67,0,3.02,MX,1595256364
74,74,ponta do sol,32.67,-17.10,73.99,60,0,1.05,PT,1595256648
103,103,klyuchi,52.27,79.17,76.57,54,0,6.11,RU,1595256654
129,129,saldanha,-33.01,17.94,73.40,49,0,4.70,ZA,1595256484
143,143,huambo,-12.78,15.74,78.04,18,0,2.59,AO,1595256663
171,171,tsabong,-26.02,22.40,72.18,19,0,7.18,BW,1595256671
191,191,ritchie,-29.04,24.60,75.20,8,0,5.82,ZA,1595256676
192,192,arkhangelsk,64.54,40.54,78.80,36,0,6.71,RU,1595256544
220,220,cidreira,-30.18,-50.21,72.91,67,0,3.18,BR,1595256684


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
4,east london,ZA,-33.02,27.91,
52,guerrero negro,MX,27.98,-114.06,
74,ponta do sol,PT,32.67,-17.10,
103,klyuchi,RU,52.27,79.17,
129,saldanha,ZA,-33.01,17.94,
143,huambo,AO,-12.78,15.74,
171,tsabong,BW,-26.02,22.40,
191,ritchie,ZA,-29.04,24.60,
192,arkhangelsk,RU,64.54,40.54,
220,cidreira,BR,-30.18,-50.21,


In [11]:

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}


In [12]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 4: east london.
Closest hotel in east london is Premier Hotel Regent.
------------
Retrieving Results for Index 52: guerrero negro.
Closest hotel in guerrero negro is Hotel Los Caracoles.
------------
Retrieving Results for Index 74: ponta do sol.
Closest hotel in ponta do sol is Estalagem Da Ponta Do Sol.
------------
Retrieving Results for Index 103: klyuchi.
Closest hotel in klyuchi is Vesolaya Mayovka.
------------
Retrieving Results for Index 129: saldanha.
Closest hotel in saldanha is Blue Bay Lodge.
------------
Retrieving Results for Index 143: huambo.
Closest hotel in huambo is Hotel Ekuikui I.
------------
Retrieving Results for Index 171: tsabong.
Closest hotel in tsabong is Dikukama Hotel.
------------
Retrieving Results for Index 191: ritchie.
Closest hotel in ritchie is The Cygnature Boutique Hotel.
------------
Retrieving Results for Index 192: arkhangelsk.
Closest hotel in arkhangelsk is Novotel Архангельск.
------------
Retrieving Results f

In [13]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
4,east london,ZA,-33.02,27.91,Premier Hotel Regent
52,guerrero negro,MX,27.98,-114.06,Hotel Los Caracoles
74,ponta do sol,PT,32.67,-17.10,Estalagem Da Ponta Do Sol
103,klyuchi,RU,52.27,79.17,Vesolaya Mayovka
129,saldanha,ZA,-33.01,17.94,Blue Bay Lodge
143,huambo,AO,-12.78,15.74,Hotel Ekuikui I
171,tsabong,BW,-26.02,22.40,Dikukama Hotel
191,ritchie,ZA,-29.04,24.60,The Cygnature Boutique Hotel
192,arkhangelsk,RU,64.54,40.54,Novotel Архангельск
220,cidreira,BR,-30.18,-50.21,Hotel Castelo


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))